In [1]:
import numpy as np
import sys
import os
from typing import Sequence
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

sys.path.append(os.path.abspath("../libs"))
sys.path.append(os.path.abspath("../utils"))

from gradient_descendent import gradient_descendent
from normalize import min_max_normalize, padronize
from loss_fn_tarefa2 import make_mse_loss_function, make_mae_loss_function, make_rmse_loss_function

# Ajuste de curva por otimização

## Carregar os dados

In [2]:
# Carregamento dos dados
df = pd.read_excel('../data/Trabalho2dados.xlsx')

df.head()

,x,y,z
0,-5.0,-5.0,-458.963629
1,-5.0,-3.8,-520.361381
2,-5.0,-2.6,-593.039231
3,-5.0,-1.4,-606.776605
4,-5.0,-0.2,-657.401892


## EDA dados

In [3]:
df.describe()

,x,y,z
count,81.000000,81.000000,81.000000
mean,-0.200000,-0.200000,36.990573
std,3.117691,3.117691,294.680404
min,-5.000000,-5.000000,-657.401892
25%,-2.600000,-2.600000,-42.780939
50%,-0.200000,-0.200000,59.403645
75%,2.200000,2.200000,170.599518
max,4.600000,4.600000,614.700100


In [4]:
df.isna().sum()

x    0
y    0
z    0
dtype: int64

In [5]:
df_pivot = df.pivot(index='y', columns='x', values='z')

# Extrair os componentes para o Plotly
x_axis = df_pivot.columns.values  # Eixo X
y_axis = df_pivot.index.values    # Eixo Y
z_grid = df_pivot.values        # Valores Z (grid)

fig = go.Figure()

# Adicionar a Superfície
fig.add_trace(go.Surface(
    x=x_axis,
    y=y_axis,
    z=z_grid,
    colorscale='Viridis',
    opacity=0.9,
    contours_z=dict(show=True, usecolormap=True, project_z=True, highlightcolor="white"),
    name='Superfície dos Dados'
))

# Adicionar os Pontos de Dados Originais
fig.add_trace(go.Scatter3d(
    x=df['x'],
    y=df['y'],
    z=df['z'],
    mode='markers',
    marker=dict(size=3, color='red', symbol='circle'),
    name='Pontos de Dados Originais'
))

# Melhorar o Layout
fig.update_layout(
    title=dict(text='z = f(x, y)', x=0.5),
    scene=dict(
        xaxis_title='Eixo X',
        yaxis_title='Eixo Y',
        zaxis_title='Eixo Z (Valor)'
    ),
    margin=dict(l=0, r=0, b=0, t=50)
)

fig.show()

In [6]:
X = np.column_stack([df['x'] ** 3, df['y'] ** 2, np.ones(len(df['x']))])

true_weights = np.linalg.inv((X.T @ X) + (X.T @ X).T) @ (2 * X.T @ df['z'])
true_weights

array([ 5.0081046 ,  5.73419945, 10.59963756])

## Calcular as funções de perda

In [7]:
features = df[['x', 'y']]
y = df['z']
features_normalized = features.copy()
features_normalized['x'] = min_max_normalize(features['x'], -1, 1)
features_normalized['y'] = min_max_normalize(features['y'], -1, 1)

features_pradonized = features.copy()
features_pradonized['x'] = padronize(features['x'])
features_pradonized['y'] = padronize(features['y'])

features_list = [features, features_normalized, features_pradonized]
features_names = ['Original', 'Min-Max Normalized', 'Padronized']

loss_fn_names = ['MSE', 'MAE', 'RMSE']
loss_fn_makers = [make_mse_loss_function, make_mae_loss_function, make_rmse_loss_function]

learning_rates = [0.001, 0.0001, 0.00001]
initial_weights = [np.zeros(3), np.array([5.0, 5.0, 8.0]), np.array([0.05, 0.05, 0.08])]
n_iterations = 5000
tolerance = 1e-6

dict_results = {}

for feature_set, feature_name in zip(features_list, features_names):
    x_data = feature_set['x'].values
    y_data = feature_set['y'].values
    z_data = y.values

    for loss_fn_name, loss_fn_maker in zip(loss_fn_names, loss_fn_makers):
        loss_function, grad_loss_function = loss_fn_maker(x_data, y_data, z_data)

        # No seu loop de treinamento, adicione:
        for lr in learning_rates:
            for initial_w in initial_weights:
                try:
                    weights, losses, n_iters = gradient_descendent(
                        initial_w, loss_function, grad_loss_function,
                        learning_rate=lr, max_iter=n_iterations, tolerance=tolerance
                    )

                    # Verificar se há overflow
                    if np.any(np.isnan(weights[-1])) or np.any(np.isinf(weights[-1])):
                        print(f"Overflow detectado para LR={lr}, Initial_W={initial_w}")
                        continue

                except Exception as e:
                    print(f"Erro para LR={lr}, Initial_W={initial_w}: {e}")
                    continue

                key = (feature_name, loss_fn_name, lr, tuple(initial_w))
                dict_results[key] = {
                    'weights': weights,
                    'losses': losses,
                    'n_iters': n_iters
                }

df_result = pd.DataFrame([
    {
        'Feature_Set': key[0],
        'Loss_Function': key[1],
        'Learning_Rate': key[2],
        'Initial_Weights': key[3],
        'Final_Weights': value['weights'][-1],
        'Final_Loss': value['losses'][-1],
        'Iterations': value['n_iters']
    }
    for key, value in dict_results.items()
])


df_result.sort_values(by='Final_Loss').groupby('Loss_Function').head(3).reset_index(drop=True)

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:86: RuntimeWarning:

overflow encountered in reduce

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\utils\loss_fn_tarefa2.py:14: RuntimeWarning:

overflow encountered in square

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\utils\loss_fn_tarefa2.py:19: RuntimeWarning:

overflow encountered in matmul

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\utils\loss_fn_tarefa2.py:19: RuntimeWarning:

invalid value encountered in matmul

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\libs\gradient_descendent.py:47: RuntimeWarning:

invalid value encountered in subtract



Overflow detectado para LR=0.001, Initial_W=[0. 0. 0.]
Overflow detectado para LR=0.001, Initial_W=[5. 5. 8.]
Overflow detectado para LR=0.001, Initial_W=[0.05 0.05 0.08]


,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Final_Weights,Final_Loss,Iterations
0,Original,MAE,0.00010,"(5.0, 5.0, 8.0)","[5.005431229629682, 5.883213629629388, 8.06220...",15.241431,5000
1,Original,MAE,0.00100,"(5.0, 5.0, 8.0)","[5.003422617283623, 5.8810261728389355, 8.1157...",15.244330,5000
2,Original,MAE,0.00100,"(0.05, 0.05, 0.08)","[5.0189366913575535, 6.2818202469128, 1.201135...",15.817841,5000
3,Original,RMSE,0.00100,"(5.0, 5.0, 8.0)","[5.006377899720806, 5.866714747314398, 8.32160...",19.348288,5000
4,Original,RMSE,0.00010,"(5.0, 5.0, 8.0)","[5.005956655896799, 5.867319533310181, 8.07845...",19.362176,5000
5,Original,RMSE,0.00100,"(0.05, 0.05, 0.08)","[5.001139518500306, 6.26873452069843, 1.410599...",20.222144,5000
6,Original,MSE,0.00010,"(5.0, 5.0, 8.0)","[5.006854213483814, 5.830160028785147, 8.95000...",373.279313,5000
7,Original,MSE,0.00001,"(5.0, 5.0, 8.0)","[5.006255230522877, 5.8761288621324, 8.1597688...",374.689397,5000
8,Original,MSE,0.00010,"(0.05, 0.05, 0.08)","[5.003120183475856, 6.11672830902677, 4.023694...",390.960242,5000


## Resultados

In [8]:
def make_function(w):
    def f(x, y):
        return w[0] * (x ** 3) + w[1] * (y ** 2) + w[2]
    return f

def plot_best_by_loss(
    df_result,
    loss_name: str,
    df,
    features_normalized,
    features_pradonized,
    title_prefix: str | None = None,
    show_original_points: bool = False
):
    """
    Filtra df_result pelo loss_name (MSE | MAE | RMSE), seleciona o melhor experimento,
    gera predições e plota a superfície original com os pontos preditos.

    Retorna um dicionário com informações do melhor experimento e métricas.
    """
    # Normalizar o nome do loss
    loss_key = loss_name.strip().upper()
    valid = {"MSE", "MAE", "RMSE"}
    if loss_key not in valid:
        raise ValueError(f"Loss inválido: {loss_name}. Use um de {sorted(valid)}")

    # Filtrar e pegar o melhor (menor Final_Loss)
    best_df = df_result[df_result["Loss_Function"].str.upper() == loss_key]
    if best_df.empty:
        raise ValueError(f"Nenhum resultado encontrado para {loss_key}")
    best_row = best_df.loc[best_df["Final_Loss"].idxmin()]

    best_w = best_row["Final_Weights"]
    feature_set_used = best_row["Feature_Set"]

    print(f"Melhor resultado {loss_key}:")
    print(f"Feature Set: {feature_set_used}")
    print(f"Learning Rate: {best_row['Learning_Rate']}")
    print(f"Final Loss: {best_row['Final_Loss']}")
    print(f"Pesos: {best_w}")

    # Escolher os dados corretos para alimentar o modelo (mesma transformação usada no treino)
    if feature_set_used == "Min-Max Normalized":
        x_data = features_normalized["x"].values
        y_data = features_normalized["y"].values
    elif feature_set_used == "Padronized":
        x_data = features_pradonized["x"].values
        y_data = features_pradonized["y"].values
    else:
        x_data = df["x"].values
        y_data = df["y"].values

    # Função de predição com os melhores pesos
    f_pred = make_function(best_w)

    # Predições para todos os pontos do dataset
    predictions = [f_pred(x, y) for x, y in zip(x_data, y_data)]

    # Superfície original (grid a partir dos dados)
    df_pivot = df.pivot(index="y", columns="x", values="z")
    x_axis = df_pivot.columns.values
    y_axis = df_pivot.index.values
    z_grid = df_pivot.values

    fig = go.Figure()

    # Superfície original
    fig.add_trace(go.Surface(
        x=x_axis,
        y=y_axis,
        z=z_grid,
        colorscale="Viridis",
        opacity=0.7,
        name="Dados Originais"
    ))

    # Pontos originais
    if show_original_points:
        fig.add_trace(go.Scatter3d(
            x=df["x"],
            y=df["y"],
            z=df["z"],
            mode="markers",
            marker=dict(size=4, color="red", symbol="circle"),
            name="Pontos Originais"
        ))

    # Pontos preditos
    fig.add_trace(go.Scatter3d(
        x=df["x"],
        y=df["y"],
        z=predictions,
        mode="markers",
        marker=dict(size=5, color="blue", symbol="circle"),
        name="Predições"
    ))

    # Métricas
    z_true = df["z"].values
    mse = float(np.mean((z_true - predictions) ** 2))
    mae = float(np.mean(np.abs(z_true - predictions)))
    rmse = float(np.sqrt(mse))

    title_main = title_prefix or "Ajuste de Curva"
    fig.update_layout(
        title=f"{title_main} - {loss_key} | Feature: {feature_set_used} | LR: {best_row['Learning_Rate']} | Loss: {best_row['Final_Loss']:.6f}",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        margin=dict(l=0, r=0, b=0, t=60),
        legend=dict(x=0.02, y=0.98)
    )
    fig.show()

    return {
        "loss": loss_key,
        "best_row": best_row,
        "weights": best_w,
        "feature_set_used": feature_set_used,
        "predictions": np.array(predictions),
        "metrics": {"MSE": mse, "MAE": mae, "RMSE": rmse},
        "fig": fig
    }


### MSE

In [9]:
_ = plot_best_by_loss(df_result, "MSE", df, features_normalized, features_pradonized)

Melhor resultado MSE:
Feature Set: Original
Learning Rate: 0.0001
Final Loss: 373.2793132812906
Pesos: [5.00685421 5.83016003 8.95000686]


In [10]:
df_result_mse = df_result[df_result['Loss_Function'] == 'MSE'].sort_values(by='Final_Loss').reset_index(drop=True)
df_result_mse

,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Final_Weights,Final_Loss,Iterations
0,Original,MSE,0.00010,"(5.0, 5.0, 8.0)","[5.006854213483814, 5.830160028785147, 8.95000...",373.279313,5000
1,Original,MSE,0.00001,"(5.0, 5.0, 8.0)","[5.006255230522877, 5.8761288621324, 8.1597688...",374.689397,5000
2,Original,MSE,0.00010,"(0.05, 0.05, 0.08)","[5.003120183475856, 6.11672830902677, 4.023694...",390.960242,5000
3,Original,MSE,0.00010,"(0.0, 0.0, 0.0)","[5.003082466001028, 6.119622938120117, 3.97393...",391.246879,5000
4,Original,MSE,0.00001,"(0.05, 0.05, 0.08)","[5.0007324469003, 6.299974254707189, 0.8735585...",413.367443,5000
5,Original,MSE,0.00001,"(0.0, 0.0, 0.0)","[5.000676661207142, 6.30425552129886, 0.799960...",413.994477,5000
6,Padronized,MSE,0.00100,"(0.0, 0.0, 0.0)","[149.73342261588488, 53.73123486154125, -16.22...",1222.792754,5000
7,Padronized,MSE,0.00100,"(0.05, 0.05, 0.08)","[149.73342261588488, 53.73030824837407, -16.22...",1222.794798,5000
8,Padronized,MSE,0.00100,"(5.0, 5.0, 8.0)","[149.73342261588488, 53.638573544822044, -16.0...",1223.005440,5000
9,Padronized,MSE,0.00010,"(0.0, 0.0, 0.0)","[145.91433780064378, 33.4389530648235, 8.01532...",1648.219588,5000


### RMSE

In [11]:
_ = plot_best_by_loss(df_result, "RMSE", df, features_normalized, features_pradonized)

Melhor resultado RMSE:
Feature Set: Original
Learning Rate: 0.001
Final Loss: 19.34828792378882
Pesos: [5.0063779  5.86671475 8.32160523]


In [12]:
df_result_rmse = df_result[df_result['Loss_Function'] == 'RMSE'].sort_values(by='Final_Loss').reset_index(drop=True)
df_result_rmse

,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Final_Weights,Final_Loss,Iterations
0,Original,RMSE,0.00100,"(5.0, 5.0, 8.0)","[5.006377899720806, 5.866714747314398, 8.32160...",19.348288,5000
1,Original,RMSE,0.00010,"(5.0, 5.0, 8.0)","[5.005956655896799, 5.867319533310181, 8.07845...",19.362176,5000
2,Original,RMSE,0.00100,"(0.05, 0.05, 0.08)","[5.001139518500306, 6.26873452069843, 1.410599...",20.222144,5000
3,Original,RMSE,0.00100,"(0.0, 0.0, 0.0)","[5.001085717281649, 6.272863497346317, 1.33961...",20.236267,5000
4,Original,RMSE,0.00001,"(5.0, 5.0, 8.0)","[4.995586925515525, 5.280559234597961, 8.01890...",20.853941,5000
5,Original,RMSE,0.00010,"(0.05, 0.05, 0.08)","[4.9788627380404815, 5.096876678813543, 0.4139...",25.798080,5000
6,Original,RMSE,0.00010,"(0.0, 0.0, 0.0)","[4.978107433531541, 5.061778956153373, 0.33450...",26.125770,5000
7,Original,RMSE,0.00001,"(0.05, 0.05, 0.08)","[2.7030976765373502, 0.24759111160694466, 0.09...",150.496257,5000
8,Original,RMSE,0.00001,"(0.0, 0.0, 0.0)","[2.6559520047267386, 0.19638731870839835, 0.01...",153.124297,5000
9,Padronized,RMSE,0.00100,"(5.0, 5.0, 8.0)","[14.342249145471948, 6.113017673298784, 8.4224...",266.051589,5000


### MAE

In [13]:
_ = plot_best_by_loss(df_result, "MAE", df, features_normalized, features_pradonized)

Melhor resultado MAE:
Feature Set: Original
Learning Rate: 0.0001
Final Loss: 15.241431139520667
Pesos: [5.00543123 5.88321363 8.06220741]


In [14]:
df_result_mae = df_result[df_result['Loss_Function'] == 'MAE'].sort_values(by='Final_Loss').reset_index(drop=True)
df_result_mae

,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Final_Weights,Final_Loss,Iterations
0,Original,MAE,0.00010,"(5.0, 5.0, 8.0)","[5.005431229629682, 5.883213629629388, 8.06220...",15.241431,5000
1,Original,MAE,0.00100,"(5.0, 5.0, 8.0)","[5.003422617283623, 5.8810261728389355, 8.1157...",15.244330,5000
2,Original,MAE,0.00100,"(0.05, 0.05, 0.08)","[5.0189366913575535, 6.2818202469128, 1.201135...",15.817841,5000
3,Original,MAE,0.00100,"(0.0, 0.0, 0.0)","[5.0175612839501795, 6.283760987653648, 1.1426...",15.824304,5000
4,Original,MAE,0.00001,"(5.0, 5.0, 8.0)","[4.999909546667032, 5.202936296297695, 8.01398...",17.149085,5000
5,Original,MAE,0.00010,"(0.05, 0.05, 0.08)","[4.956628661727751, 4.247628246913503, 0.44755...",27.120281,5000
6,Original,MAE,0.00010,"(0.0, 0.0, 0.0)","[4.953982281480847, 4.200197629629559, 0.36843...",27.580576,5000
7,Original,MAE,0.00001,"(0.05, 0.05, 0.08)","[1.8768850646911786, 0.2965296987654183, 0.100...",146.624248,5000
8,Original,MAE,0.00001,"(0.0, 0.0, 0.0)","[1.8270456355553908, 0.2464240888889038, 0.020...",148.723283,5000
9,Padronized,MAE,0.00100,"(5.0, 5.0, 8.0)","[11.236689467848938, 7.306766666668456, 9.4895...",196.161739,5000
